## Load latest headlines

In [ ]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


In [9]:
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-06-08-16-23-06 +0000,bbc,Kenyan dies in custody after arrest for online...,https://www.bbc.com/news/articles/cwy3eqpqqnzo
1,2025-06-08-16-22-38 +0000,nypost,MAGA eyes NJ governorship as Tuesday’s primary...,https://nypost.com/2025/06/08/us-news/maga-eye...
2,2025-06-08-15-11-18 +0000,nyt,"A Colombian Senator, Miguel Uribe, Is Shot at ...",https://www.nytimes.com/2025/06/07/world/ameri...
3,2025-06-08-14-59-00 +0000,bbc,Colombia presidential hopeful shot in head at ...,https://www.bbc.com/news/articles/c9dq8yxxy8zo
4,2025-06-08-14-26-02 +0000,nypost,Musk’s father says Elon made a mistake going n...,https://nypost.com/2025/06/08/us-news/musks-fa...


## Calculate word frequencies

In [10]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(f'scores-{timestamp}.csv', index=False)
score_df.head()


C:\Users\CJ\AppData\Local\Temp\ipykernel_45268\2741963311.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
35,trump,7
28,musk,5
30,elon,4
33,going,2
20,shot,2


## Rank headlines by score

In [11]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
24,2025-06-07-17-19-14 +0000,nypost,Trump threatens Elon Musk with ‘very serious c...,https://nypost.com/2025/06/07/us-news/trump-sa...,27
4,2025-06-08-14-26-02 +0000,nypost,Musk’s father says Elon made a mistake going n...,https://nypost.com/2025/06/08/us-news/musks-fa...,26
13,2025-06-08-08-57-16 +0000,nypost,"Trump rips ‘incompetent’ Newsom, LA Mayor Bass...",https://nypost.com/2025/06/08/us-news/trump-ri...,25
6,2025-06-08-13-18-29 +0000,nypost,ABC correspondent Terry Moran suspended after ...,https://nypost.com/2025/06/08/us-news/abc-news...,24
18,2025-06-08-00-53-31 +0000,nypost,Elon Musk body-checked Treasury Secretary Scot...,https://nypost.com/2025/06/07/us-news/elon-mus...,24


## Select top headlines

In [12]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
24,27,2025-06-07-17-19-14 +0000,nypost,Trump threatens Elon Musk with ‘very serious c...,https://nypost.com/2025/06/07/us-news/trump-sa...
13,18,2025-06-08-08-57-16 +0000,nypost,"Trump rips ‘incompetent’ Newsom, LA Mayor Bass...",https://nypost.com/2025/06/08/us-news/trump-ri...
9,16,2025-06-08-09-30-00 +0000,wsj,Export controls—a major concern for industries...,https://www.wsj.com/economy/trade/export-contr...
5,16,2025-06-08-14-03-33 +0000,bbc,Gaza health workers say four killed by Israeli...,https://www.bbc.com/news/articles/c5y5wpz49xlo
18,15,2025-06-08-00-53-31 +0000,nypost,Elon Musk body-checked Treasury Secretary Scot...,https://nypost.com/2025/06/07/us-news/elon-mus...
27,14,2025-06-07-16-59-12 +0000,nypost,"Hillary, Huma and Amal walk into a bar: Newest...",https://nypost.com/2025/06/07/us-news/the-newe...
6,13,2025-06-08-13-18-29 +0000,nypost,ABC correspondent Terry Moran suspended after ...,https://nypost.com/2025/06/08/us-news/abc-news...
17,13,2025-06-08-01-00-00 +0000,wsj,Businesses are warning that constantly shiftin...,https://www.wsj.com/economy/trump-tariffs-us-e...
22,12,2025-06-07-19-48-33 +0000,nypost,Ex-NY Young Republicans leader endorsed by out...,https://nypost.com/2025/06/07/us-news/ex-ny-yo...
23,11,2025-06-07-18-07-28 +0000,nypost,Trump warns of possible military action if Ira...,https://nypost.com/2025/06/07/us-news/trump-wa...


## Next steps